In [1]:
# First make the appropriate format of data (Claim + their perspectives + their evidences)
import pandas as pd

th = 8136 # perspective ids smaller than this number are the original ones

claim = "/home/squirrel/ccg-new/projects/perspective/data/database_output/re-step1/webapp_claim.csv"
perspective = "/home/squirrel/ccg-new/projects/perspective/data/database_output/re-step1/webapp_perspective.csv"
evidence = "/home/squirrel/ccg-new/projects/perspective/data/database_output/re-step1/webapp_evidence.csv"
er = "/home/squirrel/ccg-new/projects/perspective/data/database_output/re-step1/webapp_evidencerelation.csv"
re_step1 = "/home/squirrel/ccg-new/projects/perspective/data/database_output/re-step1/webapp_restep1results.csv"

claim_df = pd.read_csv(claim)
p_df = pd.read_csv(perspective)
e_df = pd.read_csv(evidence)
step1_df = pd.read_csv(re_step1)
er_df = pd.read_csv(er)

In [18]:
# Only keep the original high quality ones
step1_df = step1_df[step1_df.p_i_3 > 0.5]
step1_df = step1_df[step1_df.perspective_id <= th]
step1_df.info()

er_df = er_df[er_df.author == "GOLD"]
er_df.info()

e_df = e_df.dropna()

p_df = p_df.dropna()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4231 entries, 19372 to 23602
Data columns (total 10 columns):
id                        4231 non-null int64
claim_id                  4231 non-null int64
perspective_id            4231 non-null int64
vote_support              4231 non-null int64
vote_leaning_support      4231 non-null int64
vote_leaning_undermine    4231 non-null int64
vote_undermine            4231 non-null int64
vote_not_valid            4231 non-null int64
p_i_5                     4231 non-null float64
p_i_3                     4231 non-null float64
dtypes: float64(2), int64(8)
memory usage: 363.6 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8122 entries, 0 to 8121
Data columns (total 6 columns):
id                8122 non-null int64
author            8122 non-null object
perspective_id    8122 non-null int64
evidence_id       8122 non-null int64
comment           0 non-null float64
anno              8122 non-null object
dtypes: float64(1), int64(3), object(

In [21]:
import json

data = []

for idx, c in claim_df.iterrows():
    cid = c.id
    c_obj = {
        "claim_id" : cid,
        "claim_title": c.title,
    }
    huge_string = ""
    pid_list = step1_df.loc[step1_df.claim_id == cid].perspective_id.unique()
    
    for pid in pid_list:
        p_query = p_df[p_df.id == pid]
        
        if len(p_query) == 0:
            continue
        p_title = p_query.iloc[0].title
        huge_string = huge_string + p_title + " "
        
        eid_query = er_df[er_df.perspective_id == pid]
        if len(eid_query) == 0:
            continue
            
        eid = eid_query.iloc[0].evidence_id
        
        e_query = e_df[e_df.id == eid]
        
        if len(e_query) > 0:
            e_content = e_query.iloc[0].content
            huge_string = huge_string + e_content + " "
        
    c_obj["related"] = huge_string
    data.append(c_obj)
    
out_path ="/home/squirrel/ccg-new/projects/perspective/data/pilot13_keyword_extraction/claim_persp_evidences.json"
with open(out_path, 'w') as fout:
    json.dump(data, fout)